In [6]:
%pip install selenium
%pip install pandas
%pip install parser-libraries



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
def set_up():
    driver = webdriver.Chrome()    
    driver.maximize_window()
    return driver

# def pop_up(driver):
#     try:
#         close_button = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,"pop_up")))
#         close_button.click()
#     except Exception:
#         print("No pop Found")

def extract_page(driver):
    details = {'Date': datetime.today().strftime('%Y-%b-%d')}
    name = []
    location = []
    founded  = []
    employees = []
    about = []
    link = []
    skill = []
    name_list = driver.find_elements('xpath',"//div[@class='employer-details col-md-10 col-sm-8 col-xs-10']//div[@class='employer-job-name']")
    location_list = driver.find_elements('xpath',"//div[@class='employer-row']//div[@class='employer-locations']")
    about_list = driver.find_elements('xpath',"//div[@class='employer-details col-md-10 col-sm-8 col-xs-10']//div[@class='employer-notes ng-binding ng-scope']")
    employeer_info = driver.find_elements('xpath',"//div[@class='employer-info']")
    skill_list = driver.find_elements('xpath',"//div[@class='employer-details col-md-10 col-sm-8 col-xs-10']//ul[@class='tags candidate-opp-keywords']")
    link_list = driver.find_elements('xpath',"//div[@class='employer-details col-md-10 col-sm-8 col-xs-10']/a")
    for i in range(len(name_list)):
        try:
            n = name_list[i].text.strip()
            name.append(n)
            details['Name'] = name
            # print(n)
        except AttributeError:
            print('No info')
        try:
            loc = location_list[i].text.strip().replace("Job available in ",'')
            # print(l)
            location.append(loc)
            details['locations'] = location
        except AttributeError:
            print('No info')
        try:

            info = employeer_info[i].text.split('•')
            # print(info,'\n')
            founded.append(info[0])
            details['Founded'] = founded
            employees.append(info[1])
            details['Employees'] = employees
        except AttributeError:
            print("No Info")
        try:
            s = skill_list[i].text.replace('\n',', ')
            skill.append(s)
            details['Skill'] = skill
            # print(details['Skills'])
        except AttributeError:
            print("No info")
        
        try:
            a = about_list[i].text
            about.append(a)
            # details['about'] = abt
        except (IndexError,AttributeError) as e:
            about.append("No Info")
            # details['about'] = abt
        finally:
            details['About'] = about

        try:
            l = link_list[i].get_attribute('href')
            link.append(l)
            details['Link'] = link
        except AttributeError:
            print("No Info")
            
    return details

def navigate_page(driver,total_pages):
    total_job = pd.DataFrame()
    
    for page in range(total_pages):
        details = extract_page(driver)
        df = pd.DataFrame(details)
        total_job = pd.concat([total_job,df],axis =0)
        
        try:
            page_nav_button = driver.find_element("xpath","//div[@class='pagination ng-scope']/li[@ng-click='nextPage()']")
            page_nav_button.click()
            time.sleep(10)
        except Exception:
            print("No next page found")
            break
    return total_job


url = 'https://www.instahyre.com/python-jobs'
driver = set_up()
driver.get(url)
time.sleep(5)
total_pages = 15

job_list= navigate_page(driver,total_pages)

job_list.to_csv('Pythonjobs.csv',index=False)

No next page found
